### - **Author : Joris LAMAILLARD-NOIREN**
### - **Email : joris.larmaillard--noiren@efrei.net**
### - **Date : Monday, September 16th of 2024**

# Introduction

This notebook is designed to explore and analyze the dataset at hand in order to identify key trends and insights. Initially, we will focus on examining the structure and contents of the data, ensuring we understand the types of information available and their potential value. This exploration phase will help us determine which visualizations and plots are most suitable for showcasing relevant patterns.

By carefully inspecting the dataset, we aim to highlight the most interesting variables and relationships, guiding us toward more meaningful and impactful data visualizations later on.

# Data Analysis

In [1]:
### Module importation ###

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import prince
from prince import ca, mca
import folium

In [2]:
### Data importation ###

df = pd.read_csv('prix-des-carburants-en-france-flux-instantane-v2.csv', delimiter=';')


df.head()

,id,latitude,longitude,cp,pop,adresse,ville,services,prix,rupture,...,carburants_indisponibles,carburants_rupture_temporaire,carburants_rupture_definitive,horaires_automate_24_24,services_service,departement,code_departement,region,code_region,horaires_jour
0,25115001,4.725800e+06,592700.000000,25115,R,Z.A. les Salines,Pouilley-les-Vignes,"{""service"": [""Toilettes publiques"", ""Laverie"",...","[{""@nom"": ""Gazole"", ""@id"": ""1"", ""@maj"": ""2024-...","[{""@nom"": ""GPLc"", ""@id"": ""4"", ""@debut"": ""2018-...",...,"GPLc,E10",NaN,GPLc;E10,Oui,"Toilettes publiques,Laverie,Relais colis,Stati...",Doubs,25,Bourgogne-Franche-Comté,27.0,Automate-24-24
1,89100001,4.818300e+06,330900.000000,89100,R,84 ROUTE DE MAILLOT,Sens,"{""service"": [""Toilettes publiques"", ""Location ...","[{""@nom"": ""Gazole"", ""@id"": ""1"", ""@maj"": ""2024-...","[{""@nom"": ""GPLc"", ""@id"": ""4"", ""@debut"": ""2023-...",...,"SP95,GPLc",NaN,SP95;GPLc,Non,"Toilettes publiques,Location de véhicule,Vente...",Yonne,89,Bourgogne-Franche-Comté,27.0,NaN
2,19300002,4.540200e+06,204200.000000,19300,R,Avenue Charles de Gaulle,Égletons,"{""service"": [""Toilettes publiques"", ""Boutique ...","[{""@nom"": ""Gazole"", ""@id"": ""1"", ""@maj"": ""2024-...","[{""@nom"": ""SP95"", ""@id"": ""2"", ""@debut"": ""2017-...",...,"SP95,E85,GPLc",NaN,SP95;E85;GPLc,Non,"Toilettes publiques,Boutique alimentaire,Bouti...",Corrèze,19,Nouvelle-Aquitaine,75.0,NaN
3,13380002,4.334100e+06,545700.000000,13380,R,89 AVENUE DE LA LIBERATION,Plan-de-Cuques,NaN,"[{""@nom"": ""Gazole"", ""@id"": ""1"", ""@maj"": ""2024-...","{""@nom"": ""SP95"", ""@id"": ""2"", ""@debut"": ""2011-0...",...,"SP95,GPLc",NaN,NaN,Non,NaN,Bouches-du-Rhône,13,Provence-Alpes-Côte d'Azur,93.0,NaN
4,54260003,4.943169e+06,561589.826094,54260,R,ROUTE D'ARRANCY,Longuyon,"{""service"": [""Station de gonflage"", ""Carburant...","[{""@nom"": ""Gazole"", ""@id"": ""1"", ""@maj"": ""2024-...","[{""@nom"": ""SP95"", ""@id"": ""2"", ""@debut"": ""2018-...",...,"SP95,GPLc,SP98",NaN,SP95;GPLc;SP98,Oui,"Station de gonflage,Carburant additivé,Piste p...",Meurthe-et-Moselle,54,Grand Est,44.0,"Automate-24-24, Lundi01.00-01.00, Mardi01.00-0..."


In order to analyse the data correctly, we'll divide the dataset into several different sets.
A subset of the dataset will be linked to a specific piece of information we wish to analyse. To do this, we'll first perform a preliminary data cleansing (on the dataset as a whole), then we'll go through the datasets

We will have the following subsets of data:
- df_price: fuel price analysis
    * Visualize price trends in different regions or cities, compare prices by fuel type, or analyze the impact of price fluctuations over specific periods.
- df_geo: geographical analysis
    * Explore regional or local disparities in fuel availability and prices.
- df_shortage: fuel availability and outage management
    * Analyze the frequency and duration of fuel unavailability
- df_availability: General fuel availability
    * Comparative analysis of fuel availability and non-availability

In [3]:
df = df.drop(columns=['horaires', 'horaires_automate_24_24', 'horaires_jour', 'services', 'services_service'])

### **Prices analysis**

In [ ]:
df_price = df[['cp', 'ville', 'latitude', 'longitude', 'departement', 'code_departement', 'geom', 'prix', 'e10_prix', 'e85_prix', 'sp95_prix', 'sp98_prix', 'gazole_prix', 'gplc_prix']]

Analysis of each variable distribution

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df_price, title='Prices report')
profile.to_file("Fuel-prices.html")

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_price['e10_prix'], name='E10'))
fig.add_trace(go.Box(y=df_price['e85_prix'], name='E85'))
fig.add_trace(go.Box(y=df_price['sp95_prix'], name='SP95'))
fig.add_trace(go.Box(y=df_price['sp98_prix'], name='SP98'))
fig.add_trace(go.Box(y=df_price['gazole_prix'], name='Gazole'))
fig.add_trace(go.Box(y=df_price['gplc_prix'], name='GPLC'))

fig.update_layout(
    title='Distribution of prices',
    yaxis_title='Prices (€)',
    xaxis_title='Fuels type'
)

fig.show()

We observe that for all fuel types there is a strong presence of outliers. Our dataset presents missing values due to several factors :
- Some stations don't sell certain types of fuel, so fuel prices are not updated. 
- Some values are simply missing, so I think they've just not been recorded, or these outlets don't sell these fuels.

Next, we'll calculate the median by department, as it's more robust for asymmetrical distributions or those containing extreme values, as in our case. For example, an isolated station with a much higher price could affect the average.

In [ ]:
price_per_department = df_price.groupby(['departement', 'code_departement'])[['sp98_prix', 'sp95_prix', 'gazole_prix', 'e10_prix', 'e85_prix', 'gplc_prix']].median().reset_index()

price_per_department

In [ ]:
### Transform the DataFrame so that it's compatible with histogram display
price_per_department_melted = price_per_department.melt(id_vars=['departement', 'code_departement'], ### Columns that won't be melted -> they'll be used as index
                                                          value_vars=['sp98_prix', 'sp95_prix', 'gazole_prix', 'e10_prix', 'e85_prix'], ### These columns will be melted into a new column
                                                          var_name='Fuels type', 
                                                          value_name='Median Price')

price_per_department_melted


In [ ]:
fig = px.bar(price_per_department_melted, 
             x='departement', 
             y='Median Price',
             color='Fuels type',
             barmode='group',  ### Displays the bars side by side
             title='Median price by department',
             labels={'departement': 'Departement', 'Median Price': 'Median Price (€/L)', 'Fuels type': 'Fuels type'})

fig.show()

Let's do some visualisation with our data displayed onto the French map.
We'll use a geoson file of France in order to use a map already split by departement, and display the corresponding data to each corresponding departement.

To do this, we need to display our map according to each type of fuel. We'll proceed as follows :
- First : sp98 and sp95
- Second : e10 and e85
- Third : diesel
- Fourth : gplc

### **Loading the geoson file**

In [ ]:
import json
with open('departements.geojson') as f:
    france_departements = json.load(f)

### **sp98 & sp95**

In [ ]:
from plotly.subplots import make_subplots

### Creating a figure with two columns
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("SP98 Price by Department", "SP95 Price by Department"),
                    specs=[[{"type": "choropleth"}, {"type": "choropleth"}]])

### sp98
fig.add_trace(
    px.choropleth(
        price_per_department,
        geojson=france_departements,
        locations='code_departement',
        featureidkey="properties.code",
        color='sp98_prix',
        hover_name='departement',
        color_continuous_scale="Plasma",
        labels={'sp98_prix': 'SP98 Price (€/L)'}
    ).data[0],
    row=1, col=1
)

### sp95
fig.add_trace(
    px.choropleth(
        price_per_department,
        geojson=france_departements,
        locations='code_departement',
        featureidkey="properties.code",
        color='sp95_prix',
        hover_name='departement',
        color_continuous_scale="Plasma",
        labels={'sp95_prix': 'SP95 Price (€/L)'}
    ).data[0],
    row=1, col=2
)

### Setting appearance and geographical boundaries
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text="SP98 vs SP95 Fuel Prices by Department", height=800)

fig.show()

As we can, most of the west departements have a lower price than the rest of France, especially l'Île-de-France.
Generally speaking, we can see that prices in France for sp98 are around 1.80€ and 1.85€ : this distribution is fairly uniform throughout the country, except for Paris, which has a median value over 1.90€ per liter.
In the case of sp95, the average price range is between 1.70€ and 1.80€ across the country, except in Paris, where the price per liter is around 2.0€.

This price difference between Paris and the rest of the country can be due to several factors:
- Higher operating costs in Paris: high rents and property taxes, higher salaries
- Less competition between stations in Paris: fewer large low-cost stations, supermarket stations are rare.
- Delivery and logistics
- Demand effect
Fuel prices in Paris are higher due to a combination of additional costs (logistics, operations, taxes) and weaker competition. In addition, local policies and the demand effect play a role. In other regions, the presence of supermarket stations and larger supermarkets reduces prices through increased competition.

### **e10 & e85**

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("E10 Price by Department", "E85 Price by Department"),
                    specs=[[{"type": "choropleth"}, {"type": "choropleth"}]])

### e10
fig.add_trace(
    px.choropleth(
        price_per_department,
        geojson=france_departements,
        locations='code_departement',
        featureidkey="properties.code",
        color='e10_prix',
        hover_name='departement',
        color_continuous_scale="Plasma",
        labels={'e10_prix':'E10 Price (€/L)'}
    ).data[0],
    row=1, col=1
)

### e85
fig.add_trace(
    px.choropleth(
        price_per_department,
        geojson=france_departements,
        locations='code_departement',
        featureidkey="properties.code",
        color='e85_prix',
        hover_name='departement',
        color_continuous_scale="Plasma",
        labels={'e85_prix':'E85 Price (€/L)'}
    ).data[0],
    row=1, col=2
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text="E10 vs E85 Fuel Prices by Department", height=800)

fig.show()


This case is impressive because we see a complete contrast in average prices between e10 and e85 superethanol. E10 ethanol is much more expensive nationwide, with an  average price of 1.70€, compared with 0.83€ for e85.
However, in both cases, there is no real significant difference from one region to another: prices are fairly uniform. This may be due to the fact that there aren't as many ethanol-powered vehicles throughout the country as there are for other types of vehicle.

### **Diesel**

In [ ]:
fig = px.choropleth(
    price_per_department,
    geojson=france_departements,
    locations='code_departement',
    featureidkey="properties.code",
    color='gazole_prix',
    hover_name='departement',
    color_continuous_scale="Plasma",
    labels={'gazole_prix':'Diesel Price (€/L)'}, 
    title='Median fuel prices by department (diesel)'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

As we can see, the price of diesel is around 1.63€ nationwide. For Corsica and Paris, we see higher costs: around 1.75€ for Corsica, and 1.80€ for Paris.

### **Gplc**

In [ ]:
fig = px.choropleth(
    price_per_department,
    geojson=france_departements,
    locations='code_departement',
    featureidkey="properties.code",
    color='gplc_prix',
    hover_name='departement',
    color_continuous_scale="Plasma",
    labels={'gplc_prix':'GPLC Price (€/L)'}, 
    title='Median fuel prices by department (gplc)'
)

fig.update_geos(fitbounds="locations",
                visible=False)

fig.show()

On average, prices in the region are around €1.0. There is no real variation in price from one département to another, except for Corsica, where the average price is around 1.30€.  This difference may be due to the cost of transporting cLPG between France and Corsica: the cost of transport is therefore included in the price per liter of cLPG.

### Geographical analysis

In this section, we look at fuel shortages on a national scale.

In [ ]:
df_shortage = df[['code_departement', 'departement', 'carburants_rupture_definitive']]

df_shortage.head()

In [ ]:
dummies = df['carburants_rupture_definitive'].str.get_dummies(sep=';')

df_shortage_with_dummies = pd.concat([df_shortage, dummies], axis=1)

df_shortage_with_dummies

In [ ]:
shortage_per_department = df_shortage_with_dummies.groupby('code_departement')[dummies.columns].sum().reset_index()


In [ ]:
shortage_per_department

In [ ]:
%matplotlib inline

### Initialize MCA model
mca = prince.MCA(n_components=2)

### Fit MCA model with data
mca = mca.fit(shortage_per_department.set_index('code_departement'))

### Transform the data
mca_results = mca.transform(shortage_per_department)

plt.scatter(mca_results[0], mca_results[1])

plt.title('')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.grid(True)
plt.show()


The vast majority of departments are centered around the origin (values close to 0 on the two main components). This indicates that they share similar characteristics in terms of permanent fuel breaks.
Departments 62 and 12 stand out strongly from the others.
Department 62 has a relatively distant position at the top of the graph. This could indicate that it has a specific fuel rupture profile, linked to certain types of fuel that are permanently broken. This means that Pas-de-Calais probably has a uniquely affected fuel type.
For department 12, it is also very far along the axis of the first component, which suggests that the permanent breaks in this department are very different from the majority of the other departments.
Some departments (such as 64, 0, 76, 44) are scattered at different positions. This shows that they share more specific break patterns, but they are not as extreme as departments 62 and 12. They probably have break patterns that depend on several fuels, but less than the two extreme cases.
Remote departments may be affected by specific local factors. This may include geographical or logistical reasons, or different supply behaviour.

In [ ]:
### Counting gas stations number for each department

total_gas_stations = df_shortage_with_dummies.groupby(['departement', 'code_departement']).size().reset_index(name='total_gas_stations')

total_gas_stations.head()

In [ ]:
### Merging DataFrames
merged_df = pd.merge(total_gas_stations, shortage_per_department, on=['code_departement'])

merged_df

In [ ]:
### Calculation of the percentage of stations with a shortage of a given fuel by department
merged_df['E10_shortage_percentage'] = merged_df['E10'] / merged_df['total_gas_stations']

merged_df['E85_shortage_percentage'] = merged_df['E85'] / merged_df['total_gas_stations']

merged_df['GPLc_shortage_percentage'] = merged_df['GPLc'] / merged_df['total_gas_stations']

merged_df['Gazole_shortage_percentage'] = merged_df['Gazole'] / merged_df['total_gas_stations']

merged_df['SP95_shortage_percentage'] = merged_df['SP95'] / merged_df['total_gas_stations']

merged_df['SP98_shortage_percentage'] = merged_df['SP98'] / merged_df['total_gas_stations']

merged_df.head()

In [ ]:
fig = px.choropleth_mapbox(
    merged_df,
    geojson=france_departements,
    locations="code_departement",
    featureidkey="properties.code",  ### GeoJSON key corresponding to departments
    color="E85_shortage_percentage",  ### Column to visualise
    hover_name="departement",  ### Display department while hovering
    hover_data={
        "E85_shortage_percentage": True,
        "E10_shortage_percentage": True,
        "total_gas_stations": True
    },
    color_continuous_scale="Reds",
    mapbox_style="carto-positron",
    zoom=5,
    center={"lat": 46.603354, "lon": 1.888334}, ### Map center
    opacity=0.5
)

fig.update_layout(mapbox_style="open-street-map",
                  title="Fuel shortages by department (E85 & E10)",
                  height=800)
fig.show()

The map above shows the definitive discontinuation rate for each département for superethanol e85, also compared with e10.
Each rate is represented as a function of the total number of stations per department no longer selling these fuels permanently.
As we can see, this fuel has a fairly high rate of permanent discontinuation across the board, with rates approaching **60%** for some departments (such as Aveyron, Haute-Loire, etc.).
On the other hand, we can see that some départements have low final breakage rates for superethanol e85: Essonne, Vaucluse and Haute-Garonne, for example. These departments are mainly located on the outskirts of major cities, which encourages the presence of all types of vehicles, and therefore a need for this fuel.
Compared with superethanol e10, the shortage rate is a little lower: between **10% and 20%**, except for Corsica, of course, because of its geographical location.
This information can help carmakers to orientate their market, and find out which type of vehicle will be most interesting for the populations living in these areas.

In [ ]:
fig = px.choropleth_mapbox(
    merged_df,
    geojson=france_departements,
    locations="code_departement",
    featureidkey="properties.code",  ### GeoJSON key corresponding to departments
    color="SP95_shortage_percentage",  ### Column to visualise
    hover_name="departement",  ### Display department while hovering
    hover_data={
        "SP95_shortage_percentage": True,
        "SP98_shortage_percentage": True,
        "total_gas_stations": True
    },
    color_continuous_scale="Reds",
    mapbox_style="carto-positron",
    zoom=5,
    center={"lat": 46.603354, "lon": 1.888334},  ### Map center
    opacity=0.5
)

fig.update_layout(mapbox_style="open-street-map",
                  title="Fuel shortages by department (SP95 & SP98)",
                  height=800)
fig.show()

In the case of unleaded 95, such a high level of permanent discontinuation can be explained by the gradual cessation of use of this fuel by private consumers. In fact, the higher the octane rating of gasoline, the better combustion is controlled, protecting your engine's performance and longevity. **Unleaded 98 therefore offers better performance and engine protection than unleaded 95**. As a result, fewer and fewer service stations will be offering this type of fuel, which has a higher ultimate breakage rate.

In [ ]:
fig = px.choropleth_mapbox(
    merged_df,
    geojson=france_departements,
    locations="code_departement",
    featureidkey="properties.code",  ### GeoJSON key corresponding to departments
    color="Gazole_shortage_percentage",  ### Column to visualise
    hover_name="departement",  ### Display department while hovering
    hover_data={
        "Gazole_shortage_percentage": True,
        "total_gas_stations": True
    },
    color_continuous_scale="Reds",
    mapbox_style="carto-positron",
    zoom=5,
    center={"lat": 46.603354, "lon": 1.888334},  ### Map center
    opacity=0.5
)

fig.update_layout(mapbox_style="open-street-map",
                  title="Fuel shortages by department (Diesel)",
                  height=800)
fig.show()

As we can see, diesel has a definitive break rate throughout the country. This is perfectly normal, given that the proportion of French vehicles running on diesel is almost **53%**.

In [ ]:
heatmap_data = merged_df[['code_departement', 'E10_shortage_percentage', 'E85_shortage_percentage', 
                          'GPLc_shortage_percentage', 'Gazole_shortage_percentage', 
                          'SP95_shortage_percentage', 'SP98_shortage_percentage']]

heatmap_data.set_index('code_departement', inplace=True)

In [ ]:

fig = px.imshow(heatmap_data.T,  ### Transpose to get the fuel types on the X axis
                labels=dict(x="Department Code", y="Fuel Type", color="Shortage (%)"),
                color_continuous_scale="Viridis",
                aspect="auto",  ### Keep the axes proportional
                title="Fuel Shortage Percentage by Department")

fig.show()

As we can see, among the list of fuels, the one with the most definitive break is GPLc, with rates reaching 80% for some departments. And, as we saw earlier, the fuel with the lowest rate of permanent rupture is diesel.

In [ ]:
### Transforming the original dataset
df_filled = df_price.merge(price_per_department,
                           on=['departement', 'code_departement'],
                           suffixes=('', '_median'),
                           how='left')

for col in ['sp98_prix', 'sp95_prix', 'gazole_prix', 'e10_prix', 'e85_prix', 'gplc_prix']:
    df_filled[col] = df_filled[col].fillna(df_filled[f'{col}_median'])

df_filled = df_filled.drop([f'{col}_median' for col in ['sp98_prix', 'sp95_prix', 'gazole_prix', 'e10_prix', 'e85_prix', 'gplc_prix']], axis=1)

In [ ]:
from folium.plugins import MarkerCluster

### Setting up the map
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

### Creating a marker cluster
marker_cluster = MarkerCluster().add_to(m)

### Add marker to cluster
for i, row in df_filled.iterrows():
    folium.Marker(
        location=row['geom'].split(','),
        popup=f"""
                {row['ville']} :
                \n E10 : {row['e10_prix']} €/L
                \n E85 : {row['e85_prix']} €/L
                \n SP5 : {row['sp95_prix']} €/L
                \n SP98 : {row['sp98_prix']} €/L
                \n Gazole : {row['gazole_prix']} €/L
                \n GPLc : {row['gplc_prix']} €/L
        """,
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(marker_cluster)

m


# Study of the distribution of gas stations

Now we'll look at the breakdown of gas stations by department. In other words, our data takes into account the road network in two categories: roads and motorways, and will enable us to see where the main sales outlets are located.

In [4]:
test = pd.crosstab(df['code_departement'], df['pop'])

### Computation for each category
pop_total = test.sum()

test

pop,A,R
code_departement,,
01,9,112
02,5,85
03,3,78
04,3,55
05,0,49
...,...,...
91,9,127
92,1,77
93,2,88


In [ ]:
fig = px.pie(
    names=pop_total.index,
    values=pop_total.values,
    title="Distribution of populations by type"
)

fig.show()

As we can see, **95.5%** of service stations are located on roadsides, which is normal in the case of towns and cities, given that most motorists travel in towns and cities: more traffic means more turnover for the station owners. In particular, there are far more needs in town than on the motorways: going to work or shopping, so there are far more service stations in town.
Whereas **4.5%** of service stations are located on the edge of France's motorways. That's a total of 453 service stations located along motorways throughout the country. Excluding holiday periods, this means fewer people passing through, and therefore less need.